In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.2 MB/s eta 0:00:00


In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline



# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")



/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Load a traditional Sentiment Analysis model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:

sentence = "The overall experience is excellent , the decor is great , but the food quality was bad  and comes in large portions ."
print(f"Sentence: {sentence}")
print()

# ABSA of "food"
aspect = "place"
inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
print(inputs)
outputs = absa_model(**inputs)
print(outputs)
probs = F.softmax(outputs.logits, dim=1)
print(probs)
probs = probs.detach().numpy()[0]
print(f"Sentiment of aspect '{aspect}' is:")
for prob, label in zip(probs, ["negative", "neutral", "positive"]):
  print(f"Label {label}: {prob}")
print()
# Sentiment of aspect 'food' is:
# Label negative: 0.0009989114478230476
# Label neutral: 0.001823813421651721
# Label positive: 0.997177243232727
"""
# ABSA of "service"
aspect = "service"
inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
outputs = absa_model(**inputs)
probs = F.softmax(outputs.logits, dim=1)
probs = probs.detach().numpy()[0]
print(f"Sentiment of aspect '{aspect}' is:")
for prob, label in zip(probs, ["negative", "neutral", "positive"]):
  print(f"Label {label}: {prob}")
print()
# Sentiment of aspect 'service' is:
# Label negative: 0.9946129322052002
# Label neutral: 0.002369985682889819
# Label positive: 0.003017079783603549
"""
# Overall sentiment of the sentence
sentiment = sentiment_model([sentence])[0]
print(f"Overall sentiment: {sentiment['label']} with score {sentiment['score']}")

Sentence: The overall experience is excellent , the decor is great , but the food quality was bad  and comes in large portions .

{'input_ids': tensor([[   1,    1,  279, 1629,  517,  269, 1390,  366,  262, 5154,  269,  426,
          366,  304,  262,  645,  607,  284,  966,  263,  699,  267,  614, 9210,
          323,    2,  470,    2,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[-1.7293, -1.5935,  3.0390]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.0083, 0.0096, 0.9821]], grad_fn=<SoftmaxBackward0>)
Sentiment of aspect 'place' is:
Label negative: 0.008343429304659367
Label neutral: 0.009556477889418602
Label positive: 0.9821001291275024

Overall sentiment: negative with score 0.7478888630

In [ ]:
import pandas as pd

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_ASTE-Data-V2/16res_test_original.csv")

In [ ]:

df2 = df2.drop('opinion', axis=1)


In [ ]:
p

In [ ]:
import ast

def convert_string_to_list(string_rep):
    return ast.literal_eval(string_rep)

In [ ]:
# Apply the function to the DataFrame column
df2['aspect'] = df2['aspect'].apply(convert_string_to_list)
df2['sentiment'] = df2['sentiment'].apply(convert_string_to_list)

In [ ]:
"""
# Function to perform ABSA on a given aspect and text
def perform_absa(text, aspect):
    inputs = absa_tokenizer(f"[CLS] {text} [SEP] {aspect} [SEP]", return_tensors="pt")
    outputs = absa_model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    probs = probs.detach().numpy()[0]
    return probs

# Function to perform Sentiment Analysis on a given text
def perform_sentiment(text):
    sentiment = sentiment_model([text])[0]
    return sentiment

# Loop over the DataFrame rows and perform ABSA and Sentiment Analysis
for index, row in df2.iterrows():
    text = row["text"]
    aspect_list = row["aspect"]

    print(f"Text: {text}")
    print()

    for aspect in aspect_list:
        aspect_sentiment_probs = perform_absa(text, aspect)
        print(f"Aspect: {aspect}, Sentiment Probabilities:")
        for prob, label in zip(aspect_sentiment_probs, ["negative", "neutral", "positive"]):
            print(f"Label {label}: {prob}")
        print()

    overall_sentiment = perform_sentiment(text)
    print(f"Overall Sentiment: {overall_sentiment['label']} with score {overall_sentiment['score']}")
    print("-------------------------------------------------")
"""

'\n# Function to perform ABSA on a given aspect and text\ndef perform_absa(text, aspect):\n    inputs = absa_tokenizer(f"[CLS] {text} [SEP] {aspect} [SEP]", return_tensors="pt")\n    outputs = absa_model(**inputs)\n    probs = F.softmax(outputs.logits, dim=1)\n    probs = probs.detach().numpy()[0]\n    return probs\n\n# Function to perform Sentiment Analysis on a given text\ndef perform_sentiment(text):\n    sentiment = sentiment_model([text])[0]\n    return sentiment\n\n# Loop over the DataFrame rows and perform ABSA and Sentiment Analysis\nfor index, row in df2.iterrows():\n    text = row["text"]\n    aspect_list = row["aspect"]\n\n    print(f"Text: {text}")\n    print()\n\n    for aspect in aspect_list:\n        aspect_sentiment_probs = perform_absa(text, aspect)\n        print(f"Aspect: {aspect}, Sentiment Probabilities:")\n        for prob, label in zip(aspect_sentiment_probs, ["negative", "neutral", "positive"]):\n            print(f"Label {label}: {prob}")\n        print()\n\n  

In [ ]:


# Function to perform ABSA on a given aspect and text
def perform_absa(text, aspect):
    inputs = absa_tokenizer(f"[CLS] {text} [SEP] {aspect} [SEP]", return_tensors="pt")
    outputs = absa_model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    probs = probs.detach().numpy()[0]
    sentiment_label = "POS" if probs[2] > probs[0] and probs[2] > probs[1] else "NEG" if probs[0] > probs[1] else "NEU"
    return sentiment_label

# Create a new column 'deberta' in the DataFrame
df2["deberta"] = ""

# Loop over the DataFrame rows and perform ABSA for each aspect
for index, row in df2.iterrows():
    text = row["text"]
    aspect_list = row["aspect"]
    aspect_sentiments = []

    for aspect in aspect_list:
        aspect_sentiment = perform_absa(text, aspect)
        aspect_sentiments.append(aspect_sentiment)

    df2.at[index, "deberta"] = aspect_sentiments




In [ ]:
df2.head(50)

,text,aspect,sentiment,deberta
0,Serves really good sushi .,[sushi],[POS],[POS]
1,Not the biggest portions but adequate .,"[portions, portions]","[NEU, NEU]","[NEU, NEU]"
2,Green Tea creme brulee is a must !,[Green Tea creme brulee],[POS],[POS]
3,It has great sushi and even better service .,"[sushi, service]","[POS, POS]","[POS, POS]"
4,The entire staff was extremely accomodating an...,[staff],[POS],[POS]
5,The owner is belligerent to guests that have a...,[owner],[NEG],[NEG]
6,Good food !,[food],[POS],[POS]
7,This is a great place to get a delicious meal .,[meal],[POS],[POS]
8,The staff is pretty friendly .,[staff],[POS],[POS]
9,The onion rings are great !,[onion rings],[POS],[POS]


In [ ]:
# Function to calculate accuracy between "sentiment" and "deberta" lists
def calculate_accuracy(sentiments, deberta_sentiments):
    correct_predictions = 0
    total_predictions = 0

    for sentiment, deberta_sentiment in zip(sentiments, deberta_sentiments):
        total_predictions += len(sentiment)

        for s, d in zip(sentiment, deberta_sentiment):
            if s == d:
                correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

# Calculate accuracy
accuracy = calculate_accuracy(df2["sentiment"], df2["deberta"])
print(f"Accuracy: {accuracy}")

Accuracy: 0.9163424124513618


In [ ]:
#df2.to_csv("/content/drive/MyDrive/ABSA work/codes/deberta/results/ASTE-Data-V2/16res_test_original.csv", index=False)

NOW BOOKS TRAIN DATA FROM DOTSA DATASET.

In [ ]:
books_df = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Books_test.csv")

In [ ]:
books_df.head(10)

,Text,Aspect,Sentiment
0,This story is one of too many heartbreaking st...,story,Mixed
1,Good Faye Kellerman ! ! ! She manages to keep ...,"Faye Kellerman, characters, it","Positive, Positive, Positive"
2,Good book on cancer -- a little longer than I ...,book,Positive
3,"An ultimately moving book that details , somet...",book,Positive
4,"Wow , this was a real tear jerker - and unfort...","way, it, this","Positive, Positive, Negative"
5,"Seriously , something 's wrong when you 're ro...","prose, book III in the Soldier Son Trilogy--Ho...","Positive, Positive, Positive, Mixed, Negative,..."
6,Damaged . The Hearbreaking True sory of a Forg...,sory,Mixed
7,Gift,Gift,Positive
8,The story is horrifying . Cathy Glass is a sup...,"story, Cathy Glass","Negative, Positive"
9,Robin Hobb I think has a severe love-hate rela...,"stories, Robin Hobb","Positive, Positive"


In [ ]:
def perform_absa(text, aspect):
    inputs = absa_tokenizer(f"[CLS] {text} [SEP] {aspect} [SEP]", return_tensors="pt")
    outputs = absa_model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    probs = probs.detach().numpy()[0]
    sentiment_label = "Mixed" if probs[0] > 0.4 and probs[1] > 0.4 and probs[2] > 0.4 else "Positive" if probs[2] > probs[0] and probs[2] > probs[1] else "Negative"
    return sentiment_label

# Create a new column 'deberta' in the DataFrame
books_df["deberta"] = ""

# Loop over the DataFrame rows and perform ABSA for each aspect
for index, row in books_df.iterrows():
    text = row["Text"]
    aspect_list = row["Aspect"].split(", ")
    aspect_sentiments = []

    for aspect in aspect_list:
        aspect_sentiment = perform_absa(text, aspect)
        aspect_sentiments.append(aspect_sentiment)

    books_df.at[index, "deberta"] = ", ".join(aspect_sentiments)



In [ ]:
books_df.head(50)

,Text,Aspect,Sentiment,deberta
0,This story is one of too many heartbreaking st...,story,Mixed,Negative
1,Good Faye Kellerman ! ! ! She manages to keep ...,"Faye Kellerman, characters, it","Positive, Positive, Positive","Positive, Positive, Positive"
2,Good book on cancer -- a little longer than I ...,book,Positive,Positive
3,"An ultimately moving book that details , somet...",book,Positive,Positive
4,"Wow , this was a real tear jerker - and unfort...","way, it, this","Positive, Positive, Negative","Positive, Positive, Positive"
5,"Seriously , something 's wrong when you 're ro...","prose, book III in the Soldier Son Trilogy--Ho...","Positive, Positive, Positive, Mixed, Negative,...","Negative, Negative, Negative, Negative, Negati..."
6,Damaged . The Hearbreaking True sory of a Forg...,sory,Mixed,Positive
7,Gift,Gift,Positive,Negative
8,The story is horrifying . Cathy Glass is a sup...,"story, Cathy Glass","Negative, Positive","Negative, Positive"
9,Robin Hobb I think has a severe love-hate rela...,"stories, Robin Hobb","Positive, Positive","Negative, Negative"


In [ ]:
def calculate_accuracy(sentiments, deberta_sentiments):
    correct_predictions = 0
    total_predictions = 0

    for sentiment, deberta_sentiment in zip(sentiments, deberta_sentiments):
        sentiment_list = sentiment.split(", ")
        deberta_list = deberta_sentiment.split(", ")

        total_predictions += len(sentiment_list)

        for s, d in zip(sentiment_list, deberta_list):
            if s == d:
                correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

# Calculate accuracy
accuracy = calculate_accuracy(books_df["Sentiment"], books_df["deberta"])
print(f"Accuracy: {accuracy}")

Accuracy: 0.7472826086956522


In [ ]:
#books_df.to_csv("/content/drive/MyDrive/ABSA work/codes/deberta/results/dotsa/new_Books_test.csv", index=False)

**NOW MAMS TEST DATA**




In [ ]:
import pandas as pd

In [ ]:
mams_df = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_MAMS/ACSA_test.csv")

In [ ]:
mams_df.head(10)

,sentence,aspect_category,polarity
0,"We went again and sat at the bar this time, I ...",place,neutral
1,"We went again and sat at the bar this time, I ...",food,negative
2,"The food was good, but it's not worth the wait...",food,positive
3,"The food was good, but it's not worth the wait...",service,negative
4,Waiter took our drink order and then we didn't...,staff,negative
5,Waiter took our drink order and then we didn't...,miscellaneous,neutral
6,It does get crowded with plenty of blue shirte...,food,negative
7,It does get crowded with plenty of blue shirte...,miscellaneous,neutral
8,"After hearing all of the specials, you would t...",food,neutral
9,"After hearing all of the specials, you would t...",price,negative


In [ ]:
# Function to perform ABSA for a given sentence and aspect
# Function to perform ABSA on a given aspect and text
def perform_absa(text, aspect):
    inputs = absa_tokenizer(f"[CLS] {text} [SEP] {aspect} [SEP]", return_tensors="pt")
    outputs = absa_model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    probs = probs.detach().numpy()[0]
    sentiment_label = "positive" if probs[2] > probs[0] and probs[2] > probs[1] else "negative" if probs[0] > probs[1] else "neutral"
    return sentiment_label

# Create a new column 'predicted_polarity' in the DataFrame
mams_df["deberta"] = ""

# Perform ABSA for each sentence and aspect in the DataFrame
for index, row in mams_df.iterrows():
    sentence = row["sentence"]
    aspect = row["aspect_category"]
    predicted_sentiment = perform_absa(sentence, aspect)
    mams_df.at[index, "deberta"] = predicted_sentiment



In [ ]:
mams_df.head(50)

,sentence,aspect_category,polarity,deberta
0,"We went again and sat at the bar this time, I ...",place,neutral,neutral
1,"We went again and sat at the bar this time, I ...",food,negative,negative
2,"The food was good, but it's not worth the wait...",food,positive,positive
3,"The food was good, but it's not worth the wait...",service,negative,negative
4,Waiter took our drink order and then we didn't...,staff,negative,negative
5,Waiter took our drink order and then we didn't...,miscellaneous,neutral,neutral
6,It does get crowded with plenty of blue shirte...,food,negative,neutral
7,It does get crowded with plenty of blue shirte...,miscellaneous,neutral,neutral
8,"After hearing all of the specials, you would t...",food,neutral,neutral
9,"After hearing all of the specials, you would t...",price,negative,negative


In [ ]:
def calculate_accuracy(original_polarity, predicted_polarity):
    correct_predictions = sum(original_polarity == predicted_polarity)
    total_predictions = len(original_polarity)
    accuracy = correct_predictions / total_predictions
    return accuracy


In [ ]:
# Assuming the DataFrame "mams_df" already contains the "predicted_polarity" column
# Calculate accuracy
accuracy = calculate_accuracy(mams_df["polarity"], mams_df["deberta"])
print(f"Accuracy: {accuracy}")


Accuracy: 0.7735849056603774


In [ ]:
# Assuming 'column_name' is the name of the column you want to delete
mams_df.drop('predicted_polarity', axis=1, inplace=True)


KeyError: ignored

In [ ]:
#mams_df.to_csv("/content/drive/MyDrive/ABSA work/codes/deberta/results/MAMS/ACSA_test.csv", index=False)

**NOW YASO TEST DATA**

In [ ]:
yaso_df = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_YASO/TSA-MD_test.csv")

In [ ]:
yaso_df.head()

In [ ]:
def perform_absa(text, aspect):
    inputs = absa_tokenizer(f"[CLS] {text} [SEP] {aspect} [SEP]", return_tensors="pt")
    outputs = absa_model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    predicted_sentiment = "positive" if probs[0][1] > probs[0][0] else "negative"
    return predicted_sentiment

# Create a new column 'predicted_polarity' in the DataFrame
yaso_df["deberta"] = ""



# Perform ABSA for each sentence and aspect in the DataFrame
for index, row in yaso_df.iterrows():
    sentence = row["sentence"]
    aspect = row["target_text"]

    # Skip rows with "Target Not Found" aspect
    if aspect == "Target Not Found":
        continue

    predicted_sentiment = perform_absa(sentence, aspect)
    yaso_df.at[index, "deberta"] = predicted_sentiment



In [ ]:
yaso_df.head(50)

In [ ]:
def calculate_accuracy(original_polarity, predicted_polarity):
    correct_predictions = sum(original_polarity == predicted_polarity)
    total_predictions = len(original_polarity)
    accuracy = correct_predictions / total_predictions
    return accuracy

In [ ]:
# Assuming the DataFrame "mams_df" already contains the "predicted_polarity" column
# Calculate accuracy
accuracy = calculate_accuracy(yaso_df["sentiment"], yaso_df["deberta"])
print(f"Accuracy: {accuracy}")

In [ ]:
yaso_df.to_csv("/content/drive/MyDrive/ABSA work/codes/deberta/results/YASO/TSA-MD_test.csv", index=False)